# Dependencias

In [132]:
%pip install opencv-contrib-python
%pip install plotly==5.24.1
import cv2
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
import scipy.signal as signal

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


# Definiciones

In [133]:
# Carga el video
nombre_video = 'yoyo.mp4'
cap = cv2.VideoCapture('Videos/' + nombre_video) # Local
if not cap.isOpened():
  cap = cv2.VideoCapture('/content/' + nombre_video) # Colab


# Medidas tomadas de pixeles y metros
medida_metros = 0.10

if nombre_video == 'yoyoCL.mp4':
  medida_pixeles = 60 # Cantidad de pixeles en 0.1 metros
  # Definir rectangulos a trackear segun el primer frame
  bboxCentro = (465, 70, 73, 78)
  bboxBorde = (511, 83, 20, 21) 
elif nombre_video == 'yoyo.mp4':
  medida_pixeles = 71
  # Definir rectangulos a trackear segun el primer frame
  bboxCentro = (534, 155, 11, 13)
  bboxBorde = (557, 174, 15, 16)
const_XY = medida_metros/medida_pixeles

# Variables para guardar las posiciones y tiempos
positionsCentro = []
positionsCentro_m = []
positionsBorde = []
positionsBorde_m = []
fps = cap.get(cv2.CAP_PROP_FPS)  # Obtiene los FPS del video
frame_duration = 1.0 / fps  # Duración de cada frame en segundos # Si hay error acá, revisar video
frame_count = 0

# Trackeo

In [134]:
def calcularPosiciones(frame, bbox, positions, positions_m, number):
    # Calcular centro
    (x, y, w, h) = [int(v) for v in bbox]
    center_x = x + w // 2
    center_y = y + h // 2
    # Dibujar bounding box
    cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
    # Dibuja centro
    cv2.circle(frame, (center_x, center_y), 5, (0, 0, 255), -1)

    # Guarda posiciones
    current_time = frame_count * frame_duration
    positions.append((current_time, center_x, cap.get(cv2.CAP_PROP_FRAME_HEIGHT) - center_y))
    positions_m.append((current_time, center_x * const_XY, (cap.get(cv2.CAP_PROP_FRAME_HEIGHT) - center_y) * const_XY))

In [135]:
# Inicializar bounding boxes
bbox1 = bboxCentro
bbox2 = bboxBorde
# Crear trackers
trackerCentro = cv2.TrackerCSRT_create()
trackerBorde = cv2.TrackerCSRT_create()
# Leer primer frame
ret, frame = cap.read()
if not ret:
    print("No se pudo leer el video.")
    cap.release()
    cv2.destroyAllWindows()
# Inicializar trackers
ok1 = trackerCentro.init(frame, bbox1) # Si hay error acá, compilar definiciones
ok2 = trackerBorde.init(frame, bbox2)

while True:
    # Lee el siguiente frame
    ret, frame = cap.read()

    if not ret:
        break

    # Actualiza ambos trackers
    ok1, bbox1 = trackerCentro.update(frame)
    ok2, bbox2 = trackerBorde.update(frame)
    if ok1 and ok2:
        calcularPosiciones(frame, bbox1, positionsCentro, positionsCentro_m, 1)
        calcularPosiciones(frame, bbox2, positionsBorde, positionsBorde_m, 2)

    # Muestra el frame con el seguimiento SOLO LOCAL, NO COLAB
    #display_frame = cv2.resize(frame, (400, 1000))
    #cv2.imshow('Object Tracking', display_frame)

    # Rompe el bucle si se presiona la tecla 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    # Incrementa el contador de frames
    frame_count += 1

# Libera la captura y cierra las ventanas
cap.release()
cv2.destroyAllWindows()

# Datos

In [136]:
def filtrar_dataframe(dfp, x, y, margen):
    mediasX =  dfp[x].rolling(window=margen, center=True).mean()
    mediasY =  dfp[y].rolling(window=margen, center=True).mean()
    desviosX = dfp[x].rolling(window=margen, center=True).std()
    desviosY = dfp[y].rolling(window=margen, center=True).std()
    # Filtrar posiciones que están fuera de la media +/- 2 desvios estándar
    dfFilt = dfp[
        (dfp[x] >= mediasX - 1.5 * desviosX) &
        (dfp[x] <= mediasX + 1.5 * desviosX) &
        (dfp[y] >= mediasY - 1.5 * desviosY) &
        (dfp[y] <= mediasY + 1.5 * desviosY)
    ]
    return dfFilt

def crear_dataframe(positions):
    # Crea un DataFrame con las posiciones en funcion del tiempo que capturo el codigo de opencv
    df = pd.DataFrame(positions, columns=['Time (sec)', 'X', 'Y'])

    # Calcula las diferencias entre posiciones y tiempos, lo agrega al dataframe
    #diff: es una funcion que calcula la diferencia entre los elementos consecutivos, por ejemplo de tiempo en este caso.
    #fillna(0): remplaza los elementos vacios por un 0 para que no tire error.
    df['Delta_Time'] = df['Time (sec)'].diff().fillna(0)

    # Filtrar y recalcular
    #df = filtrar_dataframe(df, 'X', 'Y', 15)
    df['Delta_Time'] = df['Time (sec)'].diff().fillna(0)
    df['Delta_X'] = df['X'].diff().fillna(0)
    df['Delta_Y'] = df['Y'].diff().fillna(0)

    # Calcula la velocidad en X y Y a partir de las diferencias en posicion y el cambio en el tiempo, lo agrega al dataframe
    df['Speed_X'] = df['Delta_X'] / df['Delta_Time']
    df['Speed_Y'] = df['Delta_Y'] / df['Delta_Time']
    # Reemplaza inf y -inf por 0 en las velocidades en x e y (por si se va al infinito por dividir por cero por ejemplo)
    df['Speed_X'] = df['Speed_X'].replace([np.inf, -np.inf], 0)
    df['Speed_Y'] = df['Speed_Y'].replace([np.inf, -np.inf], 0)
    # Calcula la velocidad total, lo agrega al dataframe
    df['Speed'] = np.sqrt(df['Speed_X']**2 + df['Speed_Y']**2)
    # Reemplaza inf y -inf por 0 en la velocidad total
    df['Speed'] = df['Speed'].replace([np.inf, -np.inf], 0)

    # Calcular la aceleración en X e Y
    df['Acceleration_X'] = df['Speed_X'].diff() / df['Delta_Time']
    df['Acceleration_Y'] = df['Speed_Y'].diff() / df['Delta_Time']
    # Reemplaza inf y -inf por 0 en las aceleraciones en x e y
    df['Acceleration_X'] = df['Acceleration_X'].replace([np.inf, -np.inf], 0)
    df['Acceleration_Y'] = df['Acceleration_Y'].replace([np.inf, -np.inf], 0)
    # Calcula la aceleración total, lo agrega al dataframe
    df['Acceleration'] = np.sqrt(df['Acceleration_X']**2 + df['Acceleration_Y']**2)
    # Reemplaza inf y -inf por 0 en la aceleración total
    df['Acceleration'] = df['Acceleration'].replace([np.inf, -np.inf], 0)

    # Filtro Savitzky-Golay
    window_size = 26  # Adjust window size as needed
    polynomial_order = 2  # Adjust polynomial order as needed
    df['X'] = signal.savgol_filter(df['X'], window_size, polynomial_order)
    df['Y'] = signal.savgol_filter(df['Y'], window_size, polynomial_order)
    df['Speed_X'] = signal.savgol_filter(df['Speed_X'], window_size, polynomial_order)
    df['Speed_Y'] = signal.savgol_filter(df['Speed_Y'], window_size, polynomial_order)
    df['Speed'] = signal.savgol_filter(df['Speed'], window_size, polynomial_order)
    df['Acceleration_X'] = signal.savgol_filter(df['Acceleration_X'], window_size, polynomial_order)
    df['Acceleration_Y'] = signal.savgol_filter(df['Acceleration_Y'], window_size, polynomial_order)
    df['Acceleration'] = signal.savgol_filter(df['Acceleration'], window_size, polynomial_order)

    # Separar aceleracion y velocidad para eliminar ruido
    df_speed = df[['Time (sec)', 'Speed_X', 'Speed_Y', 'Speed']][df['Speed'] != 0].copy()
    df_speed = filtrar_dataframe(df_speed, 'Speed_X', 'Speed_Y', 6)
    df_acceleration = df[['Time (sec)', 'Acceleration_X', 'Acceleration_Y', 'Acceleration']][df['Acceleration'] != 0].copy()
    df_acceleration = filtrar_dataframe(df_acceleration, 'Acceleration_X', 'Acceleration_Y', 20)

    return df, df_speed, df_acceleration

def crear_dataframeRelativo(dfA, dfB):
    # Combina los DataFrames
    dfCombined = pd.merge(dfA, dfB, on='Time (sec)')

    # Calcula las diferencias, velocidades y aceleraciones
    dfCombined['Delta_Time'] = dfCombined['Time (sec)'].diff().fillna(0)

    # Calcula la posición relativa
    dfCombined['Relative_Position_X'] = dfCombined['X_x'] - dfCombined['X_y']
    dfCombined['Relative_Position_Y'] = dfCombined['Y_x'] - dfCombined['Y_y']

    # Calcula cambios de posicion
    dfCombined['Delta_X'] = dfCombined['Relative_Position_X'].diff().fillna(0)
    dfCombined['Delta_Y'] = dfCombined['Relative_Position_Y'].diff().fillna(0)

    # Calcula las velocidades
    dfCombined['Relative_Speed_X'] = dfCombined['Delta_X'] / dfCombined['Delta_Time']
    dfCombined['Relative_Speed_Y'] = dfCombined['Delta_Y'] / dfCombined['Delta_Time']

    # Calcula las aceleraciones
    dfCombined['Relative_Acceleration_X'] = dfCombined['Relative_Speed_X'].diff() / dfCombined['Delta_Time']
    dfCombined['Relative_Acceleration_Y'] = dfCombined['Relative_Speed_Y'].diff() / dfCombined['Delta_Time']

    #Separar aceleración y velocidad para eliminar ruido
    dfCombined_speed = dfCombined[['Time (sec)', 'Relative_Speed_X', 'Relative_Speed_Y']]
    dfCombined_acceleration = dfCombined[['Time (sec)', 'Relative_Acceleration_X', 'Relative_Acceleration_Y']]

    return dfCombined, dfCombined_speed, dfCombined_acceleration

In [137]:
# Creamos los DataFrames para los puntos
dfCentro, dfCentro_speed, dfCentro_acceleration = crear_dataframe(positionsCentro_m)
dfBorde, dfBorde_speed, dfBorde_acceleration = crear_dataframe(positionsBorde_m)
dfRelativoBordeCentro, dfRelativoBordeCentro_speed, dfRelativoBordeCentro_acceleration = crear_dataframeRelativo(dfBorde, dfCentro)

# Guardamos los DataFrames actualizados en archivos CSV (es para chequear despues si todo esta en orden nomas)
dfCentro.to_csv('DataFrameA.csv', index=False, float_format='%.6f')
dfBorde.to_csv('DataFrameB.csv', index=False, float_format='%.6f')
dfRelativoBordeCentro.to_csv('DataFrameC.csv', index=False, float_format='%.6f')
print("Datos de posiciones y velocidades guardados en 'dataframe.csv'.")

print(dfCentro_acceleration.count())
print(dfCentro_speed.count())
#print(dfRelativoBordeCentro.count())

Datos de posiciones y velocidades guardados en 'dataframe.csv'.
Time (sec)        64
Acceleration_X    64
Acceleration_Y    64
Acceleration      64
dtype: int64
Time (sec)    98
Speed_X       98
Speed_Y       98
Speed         98
dtype: int64


# Graficos

In [138]:
pio.renderers.default = 'vscode'  # 'browser' | 'vscode' | colab

In [139]:
# Elegir dataframe a graficar
dfG = dfCentro
dfG_speed = dfCentro_speed
dfG_acceleration = dfCentro_acceleration

# Crear figura y ejes
# 4 filas, 2 columnas
fig = make_subplots(rows=4, cols=2, subplot_titles=("Trayectoria del Objeto", "Velocidad del Objeto", "Posición X / Tiempo", "Posición Y / Tiempo","Velocidad X / Tiempo", "Velocidad Y / Tiempo", "Aceleración X / Tiempo", "Aceleración Y / Tiempo"))


# Grafica la trayectoria
fig.update_xaxes(range=[0.6,1.1], row=1, col=1)
fig.add_trace(go.Scatter(x=dfG['X'], y=dfG['Y'], mode='lines+markers', marker=dict(color='blue')), row=1, col=1)

# Grafica la velocidad
fig.add_trace(go.Scatter(x=dfG_speed['Time (sec)'], y=dfG_speed['Speed'], mode='lines+markers', marker=dict(color='red'), name='Velocidad Total'), row=1, col=2)

# Grafica la posición X / tiempo
fig.update_yaxes(range=[0.6,1.1], row=2, col=1)
fig.add_trace(go.Scatter(x=dfG['Time (sec)'], y=dfG['X'], mode='lines+markers', marker=dict(color='green')), row=2, col=1)

# Grafica la posición Y / tiempo
fig.add_trace(go.Scatter(x=dfG['Time (sec)'], y=dfG['Y'], mode='lines+markers', marker=dict(color='magenta')), row=2, col=2)

# Grafica la velocidad X / tiempo
fig.update_yaxes(range=[-0.5,0.5], row=3, col=1)
fig.add_trace(go.Scatter(x=dfG_speed['Time (sec)'], y=dfG_speed['Speed_X'], mode='lines+markers', marker=dict(color='blue')), row=3, col=1)

# Grafica la velocidad Y / tiempo
fig.add_trace(go.Scatter(x=dfG_speed['Time (sec)'], y=dfG_speed['Speed_Y'], mode='lines+markers', marker=dict(color='red')), row=3, col=2)

# Grafica la aceleración X / tiempo
fig.add_trace(go.Scatter(x=dfG_acceleration['Time (sec)'], y=dfG_acceleration['Acceleration_X'], mode='lines+markers', marker=dict(color='green')), row=4, col=1)

# Grafica la aceleración Y / tiempo
fig.add_trace(go.Scatter(x=dfG_acceleration['Time (sec)'], y=dfG_acceleration['Acceleration_Y'], mode='lines+markers', marker=dict(color='purple')), row=4, col=2)

# Actualizar los títulos y ajustar el diseño
fig.update_layout(height=1000, width=1200, title_text="Análisis de Movimiento del Centro del Objeto", showlegend=False)


# Mostrar los gráficos
fig.show() # Si hay error acá, revisar el renderer

In [140]:
#Elegir dataframe a graficar
dfG = dfBorde
dfG_speed = dfBorde_speed
dfG_acceleration = dfBorde_acceleration

# Crear figura y ejes
# 4 filas, 2 columnas
fig = make_subplots(rows=4, cols=2, subplot_titles=("Trayectoria del Objeto", "Velocidad del Objeto", "Posición X / Tiempo", "Posición Y / Tiempo","Velocidad X / Tiempo", "Velocidad Y / Tiempo", "Aceleración X / Tiempo", "Aceleración Y / Tiempo"))

# Grafica la trayectoria
fig.add_trace(go.Scatter(x=dfG['X'], y=dfG['Y'], mode='lines+markers', marker=dict(color='blue')), row=1, col=1)

# Grafica la velocidad
fig.add_trace(go.Scatter(x=dfG_speed['Time (sec)'], y=dfG_speed['Speed'], mode='lines+markers', marker=dict(color='red'), name='Velocidad Total'), row=1, col=2)

# Grafica la posición X en funcion del tiempo
fig.add_trace(go.Scatter(x=dfG['Time (sec)'], y=dfG['X'], mode='lines+markers', marker=dict(color='green')), row=2, col=1)

# Grafica la posición Y / tiempo
fig.add_trace(go.Scatter(x=dfG['Time (sec)'], y=dfG['Y'], mode='lines+markers', marker=dict(color='magenta')), row=2, col=2)

# Grafica la velocidad X / tiempo
fig.add_trace(go.Scatter(x=dfG_speed['Time (sec)'], y=dfG_speed['Speed_X'], mode='lines+markers', marker=dict(color='blue')), row=3, col=1)

# Grafica la velocidad Y / tiempo
fig.add_trace(go.Scatter(x=dfG_speed['Time (sec)'], y=dfG_speed['Speed_Y'], mode='lines+markers', marker=dict(color='red')), row=3, col=2)

# Grafica la aceleración X / tiempo
fig.add_trace(go.Scatter(x=dfG_acceleration['Time (sec)'], y=dfG_acceleration['Acceleration_X'], mode='lines+markers', marker=dict(color='green')), row=4, col=1)

# Grafica la aceleración Y / tiempo
fig.add_trace(go.Scatter(x=dfG_acceleration['Time (sec)'], y=dfG_acceleration['Acceleration_Y'], mode='lines+markers', marker=dict(color='purple')), row=4, col=2)

# Actualizar los títulos y ajustar el diseño
fig.update_layout(height=1000, width=1200, title_text="Análisis de Movimiento del Borde del Objeto", showlegend=False)

# Mostrar los gráficos
fig.show() # Si hay error acá, revisar el renderer

In [141]:
# Elegir dataframe a graficar
dfG = dfRelativoBordeCentro
dfG_speed = dfRelativoBordeCentro_speed
dfG_acceleration = dfRelativoBordeCentro_acceleration

# Crear figura y ejes
# 3 filas, 2 columnas
fig = make_subplots(rows=3, cols=2, subplot_titles=("Posición X / Tiempo", "Posición Y / Tiempo","Velocidad X / Tiempo", "Velocidad Y / Tiempo", "Aceleración X / Tiempo", "Aceleración Y / Tiempo"))

# Grafica la posición X en funcion del tiempo
fig.add_trace(go.Scatter(x=dfG['Time (sec)'], y=dfG['Relative_Position_X'], mode='lines+markers', marker=dict(color='green')), row=1, col=1)

# Grafica la posición Y / tiempo
fig.add_trace(go.Scatter(x=dfG['Time (sec)'], y=dfG['Relative_Position_Y'], mode='lines+markers', marker=dict(color='magenta')), row=1, col=2)

# Grafica la velocidad X / tiempo
fig.add_trace(go.Scatter(x=dfG_speed['Time (sec)'], y=dfG_speed['Relative_Speed_X'], mode='lines+markers', marker=dict(color='blue')), row=2, col=1)

# Grafica la velocidad Y / tiempo
fig.add_trace(go.Scatter(x=dfG_speed['Time (sec)'], y=dfG_speed['Relative_Speed_Y'], mode='lines+markers', marker=dict(color='red')), row=2, col=2)

# Grafica la aceleración X / tiempo
fig.add_trace(go.Scatter(x=dfG_acceleration['Time (sec)'], y=dfG_acceleration['Relative_Acceleration_X'], mode='lines+markers', marker=dict(color='green')), row=3, col=1)

# Grafica la aceleración Y / tiempo
fig.add_trace(go.Scatter(x=dfG_acceleration['Time (sec)'], y=dfG_acceleration['Relative_Acceleration_Y'], mode='lines+markers', marker=dict(color='purple')), row=3, col=2)

# Actualizar los títulos y ajustar el diseño
fig.update_layout(height=1000, width=1200, title_text="Análisis del movimiento relativo de ambos puntos", showlegend=False)

# Mostrar los gráficos
fig.show() # Si hay error acá, revisar el renderer